# PyCOMPSs: Using objects, lists, and synchronization. Using collections. 

In this example we will see how classes and objects can be used from PyCOMPSs, and that class methods can become tasks. The example also illustrates the use of collections

## Import the PyCOMPSs library

In [ ]:
import pycompss.interactive as ipycompss

## Start the runtime  
Initialize COMPSs runtime
Parameters indicates if the execution will generate task graph, set the monitor interval and generate debug information. 

In [ ]:
import os
if 'BINDER_SERVICE_HOST' in os.environ:
    ipycompss.start(graph=True, debug=True,
                    project_xml='../xml/project.xml',
                    resources_xml='../xml/resources.xml')
else:
    ipycompss.start(graph=True, monitor=1000, debug=True)

## Importing task and arguments directionality modules 
Import task module before annotating functions or methods 

In [ ]:
from pycompss.api.api import compss_barrier
from pycompss.api.api import compss_wait_on
from pycompss.api.task import task
from pycompss.api.parameter import *

## Declaring a class

In [ ]:
%%writefile my_shaper.py 

from pycompss.api.task import task
from pycompss.api.parameter import IN

class Shape(object):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        description = "This shape has not been described yet"

    @task(returns=int, target_direction=IN)
    def area(self):
        import time
        time.sleep(4)
        return self.x * self.y
    
    @task()    
    def scaleSize(self,scale):
        import time
        time.sleep(4)
        self.x = self.x * scale
        self.y = self.y * scale
        
    @task(returns=int, target_direction=IN)
    def perimeter(self):
        import time
        time.sleep(4)
        return 2 * self.x + 2 * self.y
    
    def describe(self,text):
        self.description = text
        
    @task(target_direction=IN)
    def infoShape(self):
        import time
        time.sleep(1)
        print('Shape x=', self.x, 'y= ', self.y)
        

In [ ]:
#Operations with collections: previous to release 2.5 
@task(returns=1)
def addAll(*mylist):
    import time
    time.sleep(1)
    sum = 0
    for ll in mylist:
        sum = sum + ll
    return sum    

In [ ]:
@task(returns=int, mylist = COLLECTION_IN)
def addAll_C(mylist):
    import time
    time.sleep(4)
    sum = 0
    for ll in mylist:
        sum = sum + ll
    return sum        

In [ ]:
@task(returns=2, mylist = COLLECTION_IN, my_otherlist = COLLECTION_IN)
def addAll_C2(mylist, my_otherlist):
    import time
    time.sleep(4)
    sum = 0
    sum2 = 0
    for ll in mylist:
        sum = sum + ll
    for jj in my_otherlist: 
        sum2 = sum2 + jj
    return sum, sum2 

In [ ]:
@task (mylist = COLLECTION_INOUT)
def scale_all (mylist, scale):
    import time
    time.sleep(4)
    for ll in mylist:
        ll.x = ll.x * scale
        ll.y = ll.y * scale 

## Invoking tasks 

In [ ]:
from my_shaper import Shape

In [ ]:
my_shapes = []
my_shapes.append(Shape(100,45))
my_shapes.append(Shape(50,50))
my_shapes.append(Shape(10,100))
my_shapes.append(Shape(20,30))

In [ ]:
all_areas = []

In [ ]:
for this_shape in my_shapes:
    all_areas.append(this_shape.area())

## Synchronizing results from tasks

In [ ]:
all_areas = compss_wait_on(all_areas)
print(all_areas)

In [ ]:
rectangle = Shape(200,25)
rectangle.scaleSize(5)
area_rectangle = rectangle.area()
rectangle = compss_wait_on(rectangle)
print('X =', rectangle.x)
area_rectangle = compss_wait_on(area_rectangle)
print('Area =', area_rectangle)

## Accessing data in collections

In [ ]:
all_perimeters=[]
my_shapes.append(rectangle)
for this_shape in my_shapes:
    all_perimeters.append(this_shape.perimeter())

In [ ]:
mysum = addAll_C(all_perimeters)
mysum = compss_wait_on(mysum)
print(mysum)

In [ ]:
# Previous version without collections
# mysum = addAll(*all_perimeters)
# mysum = compss_wait_on(mysum)
# print(mysum)

## Accessing two collections

In [ ]:
all_perimeters=[]
all_areas=[]
for this_shape in my_shapes:
    all_perimeters.append(this_shape.perimeter())
    all_areas.append(this_shape.area())

In [ ]:
[my_per, my_area] = addAll_C2(all_perimeters, all_areas)
[my_per, my_area] = compss_wait_on([my_per, my_area])
print([my_per, my_area])

## Scattering data from a collection

In [ ]:
scale_all(my_shapes,2)
scaled_areas=[]
for this_shape in my_shapes:
    scaled_areas.append(this_shape.area())
    
scaled_areas = compss_wait_on(scaled_areas)
print(scaled_areas)

## Stop the runtime

In [ ]:
ipycompss.stop(sync=True)